In [7]:
# Dependencies
from urllib.parse import quote
#import gmplot
import requests  
import time
import re
import csv
import tweepy
import json
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from itertools import cycle, islice
import os
import string
import datetime
import seaborn as sns
sns.axes_style('darkgrid')
sns.set_style('darkgrid')

# Twitter API Keys
consumer_key = "TVWJaZ1J0GNIzrcL6DaTVe92b"
consumer_secret = "vYMdWen2vtC7UHtftq6CLd0oXI7PtG40eKh1gs2IIeq7mNLTQc"
access_token = "2280809480-xWf7aW6EdEg84sGeE8DiwACueGpa9DPUDltRFSF"
access_token_secret = "FbVtjlAUr7LQ6A9DOViaFzp0ISQdEoy1J7zvrEbTekESJ"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [8]:
df=pd.read_csv('tweet_ids.csv')
tweet_list=df.values.flatten().tolist()    
print(tweet_list)

[1234, 2345]


In [9]:
def analyze_jobs(job_title, request_user):
   
    #url="http://api.glassdoor.com/api/api.htm?t.p=199084&t.k=D3Ee1oQ6L9&userip=10.0.0.133&format=json&v=1&action=jobs-stats&jc=9&l=USA&admLevelRequested=1&returnJobTitles=true&returnCities=true"
    baseUrl = "http://api.glassdoor.com/api/api.htm?format=json&v=1&action=jobs-stats&l=USA&returnCities=true"
    partnerId = "199084"
    partnerKey = "D3Ee1oQ6L9"
    url = baseUrl +"&t.p="+partnerId+"&t.k="+partnerKey +"&jt="+ quote(job_title)
    print(url)
    
    headers = {'user-agent': 'Mozilla/5.0'}
    response=requests.get(url ,headers=headers).json()
    #print(json.dumps(response,indent=2))
    print(len(response["response"]["cities"]))

    citydata = response["response"]['cities']
    citydata_df= pd.DataFrame.from_dict(citydata, orient='columns')
   
    lats=citydata_df['latitude']
    lons=citydata_df['longitude']
    
    state_name = citydata_df.groupby(['stateName'])
    num_jobs = state_name['numJobs'].sum()
    job_title='Data Scienst'
    colors = list(islice(cycle(['b', 'g', 'y']), None, len(citydata_df)))
    gender_chart = num_jobs.plot(kind="bar", title=job_title + " Jobs by State in USA", color=colors)
    gender_chart.set_xlabel("States")
    gender_chart.set_ylabel("Number of jobs")

    plt.show()

#     #show the center area and map zoom size
#     gmap = gmplot.GoogleMapPlotter(37,-95,4)
#     #add points to map
#     gmap.heatmap(lats, lons)
#     #save to map
#     sample_file_name = "mymap.html"
#     gmap.draw(sample_file_name)
    
    script_dir = os.path.dirname(__file__)
    results_dir = os.path.join(script_dir, 'job_analysis/')
    file_name = "tweetout_" + job_title +".png"
    
    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)

    plt.savefig(results_dir + file_name)
    dt = datetime.datetime.now().strftime('%m/%d/%Y')
    api.update_with_media((results_dir+file_name), \
                            "Job Analysis for " +  job_title + \
                            " (" + dt + ") (Thx @" + request_user + "!!!)")
    

In [10]:
#while(True):
counter=0
while (counter==0):
    #Setup Tweepy API Authentication
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
    # Target Search Term
    target_terms = ("@jingpc job about:")
    # Variable for holding the oldest tweet
    received_tweets = api.search(target_terms, count=100, result_type="recent")
    #print(received_tweets)
    for tweet in received_tweets["statuses"]:
        print(tweet["text"])
        if tweet["id_str"] not in tweet_list:
            tweet_list.append(tweet["id_str"]) 
            tweet_text=tweet["text"]
            #print(tweet["text"])
            tweet_text1=re.split(':', tweet_text)
            print(tweet_text1)
            try:
                print(tweet_text1[1])
                job_title=tweet_text1[1]
            except:                             
                job_title=""
                continue                       
            request_user=tweet["user"]["screen_name"]
            print(request_user)
            print(job_title)
            analyze_jobs(job_title,request_user)                       
            my_df = pd.DataFrame(tweet_list)
            print(my_df)
            my_df.to_csv('tweet_ids.csv', sep='\t', encoding='utf-8', index=False)
            
    #print(tweet_list)    
    # Once tweeted, wait 180 seconds before doing anything else
    #time.sleep(300)
    counter +=1


@jingpc job about: Nurse
['@jingpc job about', ' Nurse']
 Nurse
katiezhcng
 Nurse
http://api.glassdoor.com/api/api.htm?format=json&v=1&action=jobs-stats&l=USA&returnCities=true&t.p=199084&t.k=D3Ee1oQ6L9&jt=%20Nurse
1001


NameError: name '__file__' is not defined